In [1]:
import pandas as pd

from dateutil import parser
from datetime import datetime
from dateutil.relativedelta import relativedelta

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

In [2]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [3]:
df = pd.read_sql('SELECT * FROM symbols', con=database_connection)

In [4]:
del df['index']
df

,SYMBOL,NAME OF COMPANY,SERIES,ISIN NUMBER
0,20MICRONS,20 Microns Limited,EQ,INE144J01027
1,21STCENMGM,21st Century Management Services Limited,EQ,INE253B01015
2,3IINFOTECH,3i Infotech Limited,EQ,INE748C01020
3,3MINDIA,3M India Limited,EQ,INE470A01017
4,3PLAND,3P Land Holdings Limited,EQ,INE105C01023
...,...,...,...,...
1648,ZODJRDMKJ,Zodiac JRD- MKJ Limited,EQ,INE077B01018
1649,ZOTA,Zota Health Care LImited,EQ,INE358U01012
1650,ZUARI,Zuari Agro Chemicals Limited,EQ,INE840M01016
1651,ZUARIGLOB,Zuari Global Limited,EQ,INE217A01012


In [5]:
df2 = pd.read_sql('SELECT distinct company_name FROM mainschema.marketsmojorecos',con=database_connection)

In [6]:
df2

,company_name
0,Colgate Palmol. (I)
1,Larsen & Toubro
2,Britannia Inds
3,Sahyadri Industries
4,Tata Chemicals
...,...
790,Syngene Internation.
791,ICICI Lombard Gen.
792,ICICI Securities
793,I R C T C


In [7]:
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [8]:
stripped = []
for i in df2['company_name'] :
    i = i.lower()
    i = i.replace(" ","")
    i = i.replace(".","")
    i = i.replace("(i)","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("-","")
    stripped.append(i)

In [9]:
stripped2 = []
for i in df['NAME OF COMPANY'] :
    i = i.lower()
    i = i.replace(" ","")
    i = i.replace(".","")
    i = i.replace("(i)","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("-","")
    stripped2.append(i)

In [10]:
y=0
z=0
ilist=[]
matchlist=[]
iNotFoundList=[]
for i in stripped:
    x=0
    for j in stripped2:
        Partial_Ratio = fuzz.partial_ratio(i,j)
        if Partial_Ratio > 85:
            x=x+1
            match = j
            matchratio = Partial_Ratio
    if x==1:
        #print(i,"||",match,"||",matchratio)
        ilist.append(i)
        matchlist.append(match)
        y=y+1
    else:
        iNotFoundList.append(i)
        z=z+1

In [11]:
dfinal = pd.DataFrame({'recoName':ilist, 'formalName':matchlist})

In [12]:
dNotFound = pd.DataFrame({'recoName':iNotFoundList})

In [13]:
tot = y+z
tot

795

In [14]:
df3=df

In [15]:
stripped3 = []
for i in df3['NAME OF COMPANY'] :
    i = i.lower()
    i = i.replace(" ","")
    i = i.replace(".","")
    i = i.replace("(i)","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("-","")
    stripped3.append(i)
df3['formalName'] = stripped3

In [16]:
del df3['NAME OF COMPANY']
df3

,SYMBOL,SERIES,ISIN NUMBER,formalName
0,20MICRONS,EQ,INE144J01027,20micronslimited
1,21STCENMGM,EQ,INE253B01015,21stcenturymanagementserviceslimited
2,3IINFOTECH,EQ,INE748C01020,3iinfotechlimited
3,3MINDIA,EQ,INE470A01017,3mindialimited
4,3PLAND,EQ,INE105C01023,3plandholdingslimited
...,...,...,...,...
1648,ZODJRDMKJ,EQ,INE077B01018,zodiacjrdmkjlimited
1649,ZOTA,EQ,INE358U01012,zotahealthcarelimited
1650,ZUARI,EQ,INE840M01016,zuariagrochemicalslimited
1651,ZUARIGLOB,EQ,INE217A01012,zuarigloballimited


In [17]:
dfinal = dfinal.merge(df3,on = 'formalName', how = 'inner')
dfinal

,recoName,formalName,SYMBOL,SERIES,ISIN NUMBER
0,colgatepalmol,colgatepalmoliveindialimited,COLPAL,EQ,INE259A01022
1,britanniainds,britanniaindustrieslimited,BRITANNIA,EQ,INE216A01030
2,tatachemicals,tatachemicalslimited,TATACHEM,EQ,INE092A01019
3,marutisuzuki,marutisuzukiindialimited,MARUTI,EQ,INE585B01010
4,manappuramfinance,manappuramfinancelimited,MANAPPURAM,EQ,INE522D01027
...,...,...,...,...,...
564,adanigas,adanigaslimited,ADANIGAS,EQ,INE399L01023
565,ifbindustries,ifbindustrieslimited,IFBIND,EQ,INE559A01017
566,bharatelectronics,bharatelectronicslimited,BEL,EQ,INE263A01024
567,icicilombardgen,icicilombardgeneralinsurancecompanylimited,ICICIGI,EQ,INE765G01017


In [18]:
del dfinal[' SERIES']
del dfinal[' ISIN NUMBER']
del dfinal['formalName']
dfinal

,recoName,SYMBOL
0,colgatepalmol,COLPAL
1,britanniainds,BRITANNIA
2,tatachemicals,TATACHEM
3,marutisuzuki,MARUTI
4,manappuramfinance,MANAPPURAM
...,...,...
564,adanigas,ADANIGAS
565,ifbindustries,IFBIND
566,bharatelectronics,BEL
567,icicilombardgen,ICICIGI


In [19]:
dfinal = dfinal.append(dNotFound, ignore_index = True, sort = False)
dfinal

,recoName,SYMBOL
0,colgatepalmol,COLPAL
1,britanniainds,BRITANNIA
2,tatachemicals,TATACHEM
3,marutisuzuki,MARUTI
4,manappuramfinance,MANAPPURAM
...,...,...
790,hdfcassetmngtco,NaN
791,kpittechnologies,NaN
792,dfmfoods,NaN
793,irctc,NaN


In [92]:
dfinal.to_sql(con=database_connection, name='symbolsmapped', if_exists='append')